In [1]:
!pip install --upgrade opencv-python==4.5.2.52

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 14.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.1.26
    Uninstalling opencv-python-4.1.1.26:
      Successfully uninstalled opencv-python-4.1.1.26

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import numpy as np
import cv2
import csv
from glob import glob
import matplotlib.pyplot as plt
from collections import namedtuple
from copy import deepcopy
from tqdm import tqdm
import random
random.seed(1314)
# 确认opencv的版本，如果报错，重启内核再运行本cell即可
assert cv2.__version__ > '4.5', 'Please use OpenCV 4.5 or later.'


# 1) 数据集介绍

- 数据集是同一场景下的图像对。每一对图像都对应了一组缩放因子。

In [3]:
# 解压数据集

!unzip -qo data/data230502/大视角差图像特征提取及匹配挑战赛公开数据-初赛.zip -d ./work/image-matching-challenge-iflytek
!mv work/image-matching-challenge-iflytek/┤є╩╙╜╟▓ю═╝╧ё╠╪╒ў╠с╚б╝░╞е┼ф╠Ї╒╜╚№╣л┐к╩¤╛▌-│ї╚№/* work/image-matching-challenge-iflytek/
!rm -rf work/image-matching-challenge-iflytek/┤є╩╙╜╟▓ю═╝╧ё╠╪╒ў╠с╚б╝░╞е┼ф╠Ї╒╜╚№╣л┐к╩¤╛▌-│ї╚№  


# 2) 数据集处理分析



- 关键性处理函数的定义

In [4]:
# 关键函数的定义

# 一个元组，包含相机的内参矩阵K，以及相机的外参R和T
Gt = namedtuple('Gt', ['K', 'R', 'T'])

# 一个小的ε
eps = 1e-15

src = './work/image-matching-challenge-iflytek'

def NormalizeKeypoints(keypoints, K):
    C_x = K[0, 2]
    C_y = K[1, 2]
    f_x = K[0, 0]
    f_y = K[1, 1]
    keypoints = (keypoints - np.array([[C_x, C_y]])) / np.array([[f_x, f_y]])
    return keypoints

def ComputeMaa(err_q, err_t, thresholds_q, thresholds_t):
    '''通过不同的阈值，计算一个场景的Maa 平均的平均准确度.'''
    
    assert len(err_q) == len(err_t)
    
    acc, acc_q, acc_t = [], [], []
    for th_q, th_t in zip(thresholds_q, thresholds_t):
        acc += [(np.bitwise_and(np.array(err_q) < th_q, np.array(err_t) < th_t)).sum() / len(err_q)]
        acc_q += [(np.array(err_q) < th_q).sum() / len(err_q)]
        acc_t += [(np.array(err_t) < th_t).sum() / len(err_t)]
    return np.mean(acc), np.array(acc), np.array(acc_q), np.array(acc_t)

In [5]:
def ReadpairsData(filename):
    # 1) define dict 
    covisibility_dict = {}
    # 2) 
        # 2.1 open csv file
    with open(filename) as f:
        # 2.2 read csv file
        reader = csv.reader(f, delimiter=',')
        # 2.3 load data
        for i, row in enumerate(reader):
            # Skip header.
            if i == 0:
                continue
            covisibility_dict[row[0]] = 0
    # 3) return 
    return covisibility_dict

In [6]:
def LoadCalibration_test(filename):
    '''从.csv文件中加载真值'''
    # 1) 创建变量用于输出
    calib_dict = {}
    # 2) 处理文件
        # 2.1 打开文件
    with open(filename, 'r') as f:
        # 2.2 读取数据
        reader = csv.reader(f, delimiter=',')
        # 2.3 遍历数据
        for i, row in enumerate(reader):
            # 2.3.1 Skip header.
            if i == 0:
                continue
            # 2.3.2 提取数据
                # 2.3.2.1 图像编号
            camera_id = row[0]
                # 2.3.2.2 相机内参
            K = np.array([float(v) for v in row[1].split(' ')]).reshape([3, 3])
                # 2.3.2.3 旋转矩阵
            R = np.zeros((3,3))
                # 2.3.2.4 位移向量
            T = np.zeros((3,3))
            # 2.3.3 打包变量输出
            calib_dict[camera_id] = Gt(K=K, R=R, T=T)
    
    return calib_dict 

In [23]:
pairs = ReadpairsData(f'{src}/test/pair.csv')    

calib_dict = LoadCalibration_test(f'{src}/test/calibration.csv')

images_path_dict={}
for filename in glob(f'{src}/test/images/*'):
    cur_id = os.path.basename(os.path.splitext(filename)[0])
    images_path_dict[cur_id] = filename
print(f'Loaded {len(images_path_dict)} images.')

Loaded 202 images.



# 3) SIFT算法实验

- 该比赛的目标是要得到两张照片的相对姿态，即旋转矩阵R与平移向量T
- 本项目先用比较经典的**sift算法**提取特征点并进行匹配、计算像片对的相对位姿

In [9]:
# input:
    # image: 图片
    # detector: 检测器
    # num_features: 特征点数量
# output:
    # kp[:num_features]: 关键点
    # desc[:num_features]: 描述子
def ExtractSiftFeatures(image, detector, num_features):
    '''计算图像的SIFT特征'''
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp[:num_features], desc[:num_features]

In [10]:
def ArrayFromCvKps(kps):
    '''将opencv返回的关键点转为numpy矩阵的形式'''
    
    return np.array([kp.pt for kp in kps])

In [11]:
def ComputeEssentialMatrix(F, K1, K2, kp1, kp2):
    ''''在给定校准矩阵的情况下，从基础矩阵计算本质矩阵，参赛者需要不依赖给定的内参计算基础矩阵'''
    
    # 老版本的OpenCV计算基础矩阵是返回多个，本项目使用版本为：   
    # https://opencv.org/evaluating-opencvs-new-ransacs
    assert F.shape[0] == 3, 'Malformed F?'

    # 使用 OpenCV的 recoverPose 功能计算两张照片的相对位姿R,t:
    # https://docs.opencv.org/4.5.4/d9/d0c/group__calib3d.html#gadb7d2dfcc184c1d2f496d8639f4371c0
    E = np.matmul(np.matmul(K2.T, F), K1).astype(np.float64)
    
    kp1n = NormalizeKeypoints(kp1, K1)
    kp2n = NormalizeKeypoints(kp2, K2)
    num_inliers, R, T, mask = cv2.recoverPose(E, kp1n, kp2n)

    return E, R, T

In [15]:
show_images = False
num_show_images = 1
max_pairs_per_scene = 50
verbose = False

# We use two different sets of thresholds over rotation and translation. Do not change this -- these are the values used by the scoring back-end.
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

# Instantiate the matcher.
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
num_features=5000
sift_detector = cv2.SIFT_create(num_features, contrastThreshold=-10000, edgeThreshold=-10000)

In [18]:
images_dict = {}
kp_dict = {}
desc_dict = {}
# print('Extracting features...')
for id in images_path_dict:
    images_dict[id] = cv2.cvtColor(cv2.imread(images_path_dict[id]), cv2.COLOR_BGR2RGB)
    kp_dict[id], desc_dict[id] = ExtractSiftFeatures(images_dict[id], sift_detector, 2000)
# print()

In [ ]:
F_dict={}
for counter, pair in enumerate(pairs):
    print('---read---')
    id1, id2 = pair.split('-')
    #------------------------
    cv_matches = bf.match(desc_dict[id1], desc_dict[id2])
    matches = np.array([[m.queryIdx, m.trainIdx] for m in cv_matches])
    cur_kp_1 = ArrayFromCvKps([kp_dict[id1][m[0]] for m in matches])
    cur_kp_2 = ArrayFromCvKps([kp_dict[id2][m[1]] for m in matches])

    # Filter matches with RANSAC.
    F, inlier_mask = cv2.findFundamentalMat(cur_kp_1, cur_kp_2, cv2.USAC_MAGSAC, 0.25, 0.99999, 10000)
    inlier_mask = inlier_mask.astype(bool).flatten()
    
    matches_after_ransac = np.array([match for match, is_inlier in zip(matches, inlier_mask) if is_inlier])
    inlier_kp_1 = ArrayFromCvKps([kp_dict[id1][m[0]] for m in matches_after_ransac])
    inlier_kp_2 = ArrayFromCvKps([kp_dict[id2][m[1]] for m in matches_after_ransac])

    # Compute the essential matrix.
    print('--Essential--')
    E, R, T = ComputeEssentialMatrix(F, calib_dict[id1].K, calib_dict[id2].K, inlier_kp_1, inlier_kp_2)
    F_dict[pair]=[R,T]
  

# 4) Submit

In [20]:
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

In [ ]:
for sample_id, F in F_dict.items():
        print(sample_id)
        print(FlattenMatrix(F[0]))
        print(FlattenMatrix(F[1]))

In [22]:
with open('submission.csv', 'w') as f:
    f.write('pair,rotation_matrix,translation_vector\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F[0])},{FlattenMatrix(F[1])}\n')


# 5) Ref and conclusion


[[1] 图像匹配挑战赛介绍，并以SuperGlue与SIFT特征点匹配实验对比精度](https://aistudio.baidu.com/aistudio/projectdetail/4196840?channelType=0&channel=0)


